In [ ]:
import pandas as pd
import re
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

%store -r

In [ ]:
%run PrepareData.ipynb

In [ ]:
tsla_tweets_df.head()

In [ ]:
# The purpose of this function is to prepare text for lexicon by removing special characters and symbols that are irrelavant for the tweet classification
def cleanTweet(text):
    # Remove '#' hash tag     
    text = re.sub('#', '', text) 
    #Remove @mentions 
    text = re.sub('@[A-Za-z0–9]+', '', text) 
    # Remove RT
    tweet = re.sub('RT[\s]+', '', text) 
    # Remove hyperlink
    tweet = re.sub('https?:\/\/\S+', '', text)  
    return text

In [ ]:
analyser = SentimentIntensityAnalyzer()
tsla_tweets_df["polarityScore"] = tsla_tweets_df["body"].apply(lambda tweet : analyser.polarity_scores(cleanTweet(tweet))["compound"] )
tsla_tweets_df["sentiment"] = tsla_tweets_df["polarityScore"].apply(lambda ps : 1 if(ps >=0.5) else 0 )
display(tsla_tweets_df.head())
display(tsla_tweets_df.tail())
tsla_tweets_df["sentiment"].value_counts()

In [ ]:
tsla_sentiments_df = tsla_tweets_df[["post_date","polarityScore"]].copy()
tsla_sentiments_df['date'] = tsla_sentiments_df['post_date'].dt.date
tsla_sentiments_df = tsla_sentiments_df.groupby('date').mean()
display(tsla_sentiments_df.tail())
tsla_sentiments_df.plot(kind="line")

In [ ]:
tsla_stock_values_df.head()

In [ ]:
tsla_stock_values_df["tomorrow_value"] = tsla_stock_values_df["close_value"].shift(-1)
tsla_stock_values_df["target"] = (tsla_stock_values_df["tomorrow_value"] > tsla_stock_values_df["close_value"]).astype(int)
display(tsla_stock_values_df.head())
display(tsla_stock_values_df.tail())
tsla_stock_values_df.info()